In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
from sqlalchemy import create_engine
from shapely.geometry import shape
from io import BytesIO
import base64

from dotenv import load_dotenv
import os
import json

# .env 파일을 찾아 환경 변수로 로드
load_dotenv()

# 환경 변수 가져오기
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# ▶️ MySQL 연결
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")

# ▶️ 반려동물 등록 데이터 가져오기
query = """
SELECT *
FROM companion_animal_registration
"""
df = pd.read_sql(query, engine)

# ▶️ 행정구역 명칭을 folium key와 맞추기 위해 하나로 합치기
df['region'] = df['sigungu']

# 'rank'행 추가
    # 부정지표
rank_map = {
    '관악구': 1,
    '강서구': 2,
    '도봉구': 3,
    '양천구': 4,
    '서초구': 5,
}

# 'score' 행 추가
score_map = {
    '관악구': 17,
    '강서구': 10,
    '도봉구': 9,
    '양천구': 7,
    '서초구': 7,
}

df['rank'] = df['sigungu'].map(rank_map)
df['score'] = df['sigungu'].map(score_map)

# 색깔 열 추가
# rank가 있으면 하늘색, 없으면 회색 지정
def color_map(rank):
    if pd.isna(rank):
        return 'yellow'
    else:
        return 'orange'

df['color'] = df['rank'].apply(color_map)
df

,id,sido,sigungu,dog_registered_total,cat_registered_total,total_registered,created_at,region,rank,score,color
0,1,서울,전체,537884,7278,545162,2025-07-03 04:11:20,전체,NaN,NaN,yellow
1,2,서울특별시,강남구,34402,441,34843,2025-07-03 04:11:21,강남구,NaN,NaN,yellow
2,3,서울특별시,강동구,25292,223,25515,2025-07-03 04:11:21,강동구,NaN,NaN,yellow
3,4,서울특별시,강북구,18444,141,18585,2025-07-03 04:11:21,강북구,NaN,NaN,yellow
4,5,서울특별시,강서구,33280,356,33636,2025-07-03 04:11:21,강서구,2.0,10.0,orange
...,...,...,...,...,...,...,...,...,...,...,...
247,248,경상남도,함양군,1537,3,1540,2025-07-03 04:11:22,함양군,NaN,NaN,yellow
248,249,경상남도,합천군,1704,1,1705,2025-07-03 04:11:22,합천군,NaN,NaN,yellow
249,250,제주,전체,52807,2965,55772,2025-07-03 04:11:22,전체,NaN,NaN,yellow
250,251,제주특별자치도,서귀포시,13367,466,13833,2025-07-03 04:11:22,서귀포시,NaN,NaN,yellow


In [2]:
%cd ~/eda-repo-3/

/home/park/eda-repo-3


In [ ]:
# ▶️ GeoJSON 경로 지정
geo_path = 'DATA/02. skorea_municipalities_geo_simple.json'  # 반드시 'region' 컬럼과 feature.properties.name이 일치해야 함

# ▶️ GeoJSON 로드 (시도+시군구 이름이 properties.name으로 되어 있어야 함)
with open(geo_path, 'r', encoding='utf-8') as f:
    geo = json.load(f)

# ▶️ 지도 시각화
m = folium.Map(location=[36.5, 127.5], zoom_start=8)

# 3. GeoJSON에서 각 구의 중심좌표(centroid) 추출
regions, lats, lngs = [], [], []
for feature in geo['features']:
    region_name = feature['properties']['name']
    geometry = feature['geometry']
    centroid = shape(geometry).centroid
    regions.append(region_name)
    lats.append(centroid.y)
    lngs.append(centroid.x)
geo_df = pd.DataFrame({'region': regions, 'lat': lats, 'lng': lngs})

# 4. 기존 데이터와 중심좌표 병합
df = pd.merge(df, geo_df, on='region', how='left')

# 스타일 함수 정의
def style_function(feature):
    region_name = feature['properties']['name']
    color = df.loc[df['region'] == region_name, 'color']
    if not color.empty:
        return {'fillColor': color.values[0], 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}

# GeoJson 레이어 추가
folium.GeoJson(
    geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'])
).add_to(m)

# 마커 스타일 개선 (더 눈에 잘 보이도록)
for idx, row in df.iterrows():
    if not pd.isna(row['rank']):
        folium.Marker(
            location=[row['lat'], row['lng']],
            icon=folium.DivIcon(
                html=f"""<div style="
                    font-size: 13px; 
                    color: white; 
                    background: rgba(30, 144, 255, 0.8); 
                    border-radius: 15px; 
                    padding: 6px 10px;
                    font-weight: bold;
                    text-align: center;
                    border: 2px solid white;
                    box-shadow: 0 2px 4px rgba(0,0,0,0.3);
                    white-space: nowrap;
                ">{int(row['rank'])}위 {row['region']}</div>""",
                icon_size=(80, 30),
                icon_anchor=(40, 15)
            ),
            tooltip=f"{row['region']} 순위: {int(row['rank'])}"
        ).add_to(m)

# 8. 지도 저장 또는 출력
m.save('RESULT/visualization/happy_home_folium_choropleth_5_negative.html')

# # 결과 지도 저장 또는 출력
# m.save('RESULT/visualization/happy_home_folium_choropleth.html')
m

In [ ]:

# ▶️ GeoJSON 경로 지정
geo_path = 'DATA/02. skorea_municipalities_geo_simple.json'  # 반드시 'region' 컬럼과 feature.properties.name이 일치해야 함

# ▶️ GeoJSON 로드 (시도+시군구 이름이 properties.name으로 되어 있어야 함)
with open(geo_path, 'r', encoding='utf-8') as f:
    geo = json.load(f)

# ▶️ 지도 시각화
m = folium.Map(location=[36.5, 127.5], zoom_start=8)

# 스타일 함수 정의
def style_function(feature):
    region_name = feature['properties']['name']
    color = df.loc[df['region'] == region_name, 'color']
    if not color.empty:
        return {'fillColor': color.values[0], 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}

# GeoJson 레이어 추가
folium.GeoJson(
    geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'])
).add_to(m)

# # 결과 지도 저장 또는 출력
# m.save('RESULT/visualization/happy_home_folium_choropleth.html')


# 각 구의 중심좌표를 계산하여 DataFrame에 추가하는 예시
def get_centroid(geometry):
    polygon = shape(geometry)
    centroid = polygon.centroid
    return centroid.y, centroid.x  # (lat, lng)

# 예시: df['lat'], df['lng'] 컬럼 추가
df['lat'], df['lng'] = zip(*df['geometry'].apply(get_centroid))

# folium Marker로 랭크 표시 (NaN 제외)
for idx, row in df.iterrows():
    if not pd.isna(row['rank']):
        folium.Marker(
            location=[row['lat'], row['lng']],
            icon=folium.DivIcon(html=f"""<div style="font-size: 14px; color: black; background: rgba(255,255,255,0.7); border-radius: 5px; padding: 2px 5px;">{int(row['rank'])}</div>"""),
            tooltip=f"{row['region']} 순위: {int(row['rank'])}"
        ).add_to(m)


# folium.Choropleth(
#     geo_data=geo_path,
#     data=df,
#     columns=['region', 'rank'],
#     key_on='feature.properties.name',
#     fill_color='YlOrRd',
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name='반려동물 행복 거주지 종합순위 TOP 10'
# ).add_to(m)


In [4]:
%cd ~/eda-repo-3/

/home/park/eda-repo-3


KeyError: 'lat'

In [25]:
# GeoJSON 및 데이터프레임 준비 (이미 준비된 상태라고 가정)
# geo_path, geo, df 등

# 지도 객체 생성
m = folium.Map(location=[36.5, 127.5], zoom_start=8)

# 스타일 함수 정의 (이미 작성한 코드)
def style_function(feature):
    region_name = feature['properties']['name']
    color = df.loc[df['region'] == region_name, 'color']
    if not color.empty:
        return {'fillColor': color.values[0], 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}

# GeoJson 레이어 추가
folium.GeoJson(
    geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'])
).add_to(m)

# 2. 각 지역에 랭크 값 표시 (NaN은 표시하지 않음)
for idx, row in df.iterrows():
    region = row['region']
    rank = row['rank']
    # NaN이 아니고, 좌표(위도, 경도)가 있는 경우에만 표시
    if not pd.isna(rank) and ('lat' in row) and ('lng' in row):
        folium.Marker(
            location=[row['lat'], row['lng']],
            icon=folium.DivIcon(html=f"""<div style="font-size: 14px; color: black; background: rgba(255,255,255,0.7); border-radius: 5px; padding: 2px 5px;">{int(rank)}</div>"""),
            tooltip=f"{region} 순위: {int(rank)}"
        ).add_to(m)

# 결과 지도 저장 또는 출력
# m.save('result.html')
m